# Ge/Ay 117: Problem Set 4

In [3]:
# import modules 

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import integrate
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
# import data

rawData = pd.read_csv('ps4_data.txt', header = None ,sep = ' ')
times = rawData[rawData.columns[0]].values
obliqu = rawData[rawData.columns[1]].values
obliqSig = rawData[rawData.columns[2]].values

In [5]:
def obliFit(param,t):
    A = param[0]
    B = param[1]
    P = param[2]
    return A*np.sin(((2*np.pi)/P)*t) + B

In [6]:
def negLnLikelihood(param,t,obliq,obliq_err):
    return -(np.sum(((-1/2)*np.power((obliFit(param,t)- obliq)/(obliq_err), 2))+np.log(1/(obliq_err*np.sqrt(2*np.pi)))))

param_guess=np.array([15,25,125000]) # Put your initial guess here [A, B, P]

minResult=minimize(negLnLikelihood,param_guess,args=(times,obliqu,obliqSig))

bestFits = minResult.x
invHess = minResult.hess_inv

print('Message: {}'.format(minResult.message))
print('Success?: {}\n'.format(minResult.success))
print('The covariance matrix is: \n{}\n'.format(invHess))
print('[A, B, P] = {}\n'.format(bestFits))
print('Marginalized Error Bars: {}'.format(np.sqrt(invHess.diagonal())))

Message: Optimization terminated successfully.
Success?: True

The covariance matrix is: 
[[ 8.43726305e-04  2.50152400e-05 -5.75944736e-02]
 [ 2.50152400e-05  4.08743205e-04 -6.46394709e-03]
 [-5.75944736e-02 -6.46394709e-03  2.48672321e+03]]

[A, B, P] = [9.44920844e+00 2.47346882e+01 1.23466703e+05]

Marginalized Error Bars: [2.90469672e-02 2.02173986e-02 4.98670553e+01]


In [7]:
x = times
xMod = np.linspace(0,250000)
y = obliqu


p = figure(
   tools="pan,box_zoom,reset,save", title="117 Set 4 Problem 1b",
   x_axis_label='Time (yrs)', y_axis_label='Obliquity (in degrees)'
)

p.circle(x,y)
p.line(xMod,obliFit(param_guess, xMod),color='black',line_width=2)

show(p)

In [8]:
paramErrs = np.sqrt(invHess.diagonal())

def fitError(bestFits, paramErrs, t):
    AFit = bestFits[0]
    BFit = bestFits[1]
    PFit = bestFits[2]
    AFitErr = paramErrs[0]
    BFitErr = paramErrs[1]
    PFitErr = paramErrs[2]
    return(np.sqrt(np.power(AFitErr*np.sin(2*np.pi*t/PFit), 2) + np.power(BFitErr, 2) + np.power(PFitErr*(-2*np.pi*AFit*np.cos(2*np.pi*t/PFit))/(np.power(PFit, 2)), 2)))


In [9]:
fitErrs = []
for i in range(len(times)):
    fitErrs.append(float(fitError(bestFits,paramErrs,times[[i]])))
    

In [10]:
x = times
xMod = np.linspace(0,250000)
y = obliqu
yModErrs = fitErrs
yModPart = obliFit(bestFits,x)


p2 = figure(
   tools="pan,box_zoom,reset,save", title="117 Set 4 Problem 1c",
   x_axis_label='Time (yrs)', y_axis_label='Obliquity (in degrees)'
)

err_xs = []
err_ys = []

for xs, ys, yerrs in zip(x, y, obliqSig):
    err_xs.append((xs, xs))
    err_ys.append((ys - yerrs, ys + yerrs))

p2.multi_line(err_xs, err_ys, color='purple')
p2.circle(x,y)
p2.line(xMod,obliFit(bestFits, xMod),color='black',line_width=2)

show(p2)

In [11]:
sys_guess = np.std(obliqu - obliFit(bestFits,times))
print(sys_guess)

8.12764290478983


In [12]:
def negLnLikelihood2(params,t,obliq,obliq_sig):
    param = params[:-1]
    obliq_errSys = params[-1]
    return -(np.sum(((-1/(2*(np.power(obliq_sig,2)+np.power(obliq_errSys,2))))*np.power(obliFit(param,t)-obliq,2))+np.log(1/((np.sqrt(np.power(obliq_sig,2)+np.power(obliq_errSys,2)))*np.sqrt(2*np.pi)))))

param2_guess=np.array([11,24,122600,sys_guess]) # Put your initial guess here [A, B, P, sig_sys]

minResult2=minimize(negLnLikelihood2,param2_guess,args=(times,obliqu,obliqSig))

bestFits2 = minResult2.x
invHess2 = minResult2.hess_inv

print('Message: {}'.format(minResult2.message))
print('Success?: {}\n'.format(minResult2.success))
print('The covariance matrix is: \n{}\n'.format(invHess2))
print('[A, B, P, sig_sys] = {}\n'.format(bestFits2))
print('Marginalized Error Bars: {}'.format(np.sqrt(invHess2.diagonal())))

Message: Optimization terminated successfully.
Success?: True

The covariance matrix is: 
[[ 2.54526999e-01 -1.97230162e-03 -2.21138812e+01 -2.51230148e-03]
 [-1.97230162e-03  1.35005359e-01  9.90287784e+00  1.39490669e-03]
 [-2.21138812e+01  9.90287784e+00  6.92383456e+05  3.30850250e+00]
 [-2.51230148e-03  1.39490669e-03  3.30850250e+00  6.50591288e-02]]

[A, B, P, sig_sys] = [9.95128220e+00 2.47619956e+01 1.23661814e+05 8.09301442e+00]

Marginalized Error Bars: [5.04506689e-01 3.67430754e-01 8.32095821e+02 2.55066911e-01]


In [13]:
xMod = np.linspace(0,250000)

p3 = figure(
   tools="pan,box_zoom,reset,save", title="117 Set 4 Problem 2c",
   x_axis_label='Time (yrs)', y_axis_label='Obliquity (in degrees)')

p3.line(xMod,obliFit(bestFits,xMod),color='black',line_width=2, legend='Old Best Fit')
p3.line(xMod,obliFit(bestFits2,xMod),color='green',line_width=2, legend='New Best Fit')
p3.legend.location = "top_left"

show(p3)

In [14]:
def sigTrue(bestfit, t):
    return np.sqrt(np.power(obliqSig,2)+np.power(bestfit[-1],2))
    

In [15]:
fitErrs2 = (sigTrue(bestFits2,times[[i]]))

In [16]:
x = times
xMod = np.linspace(0,250000)
y = obliqu
yModErrs2 = fitErrs2

p4 = figure(
   tools="pan,box_zoom,reset,save", title="117 Set 4 Problem 2d",
   x_axis_label='Time (yrs)', y_axis_label='Obliquity (in degrees)')


err2_xs = []
err2_ys = []

for xs, ys, yerrs in zip(x, y, yModErrs2):
    err2_xs.append((xs, xs))
    err2_ys.append((ys - yerrs, ys + yerrs))

p4.multi_line(err2_xs, err2_ys, color='pink')
p4.circle(x,y)
p4.line(xMod,obliFit(bestFits2,xMod),color='black',line_width=2)

show(p4)

pros -> faster, 
cons -> bad if multipeaked,